# Overview

Before we got our own fine-tuned LLMs. How we can make our basline LLM in a higher accuracy? One of the answers might be **Retrieval Augmented Generation(RAG)**. RAG enables LLMs reach beyond their static knowledge and grab real-time information quite easily. RAG taps into external sources, like internal company documents, to understand the context of our queries a bit better.

![](https://cdn.masto.host/sigmoidsocial/media_attachments/files/112/064/408/832/155/132/small/3f7a8bd57563298f.webp)

In this notebook, we are going to use RAG model. Retrieval-agumented generation("RAG") models combine the powers of pretrained dense retrieval(DPR) and Seq2Seq2 models. RAG models retrieve docs, pass them to a seq2seq2 model, then marginalize to generate outputs. The retriever and seq2seq modules are initlized from pretrained models, and fine-tuned jointly, allowing both retreival and generation to adapt to downstream tasks. Here are some classes inside transformers that we need to know:


# RagConfig Class

It stores the configuration of a RagModel which is used to control thr model outputs.


# Retriever Class

It used to get documents from vector queries. It retrives the documents embeddings as well as the documents contents, and it formats them to be used with a RagModel.


# RAGModel Class

RAG is a seq2seq model which encapsulates two core components: a question encoder and a generator. During a forward pass, we encode the input with the question encoder and pass it to the retriever to extract releveant context documents. The documents are then prepended to the input. Such contextualized inputs is passed to the generator.

The question encoder can be any autoencoding model, preferably **DPPQuestionEncoder** and the generator can be any **seq2seq**, preferably **BartForConditionalGeneration**. So, we can initialzed with the RAG model with any autoencoding mdoel as the question_encoder and any seq2seq model with language model heads as the generator. It inherits from **PreTrainedModel**, and it is also a PyTorch **torch.nn.Module** subclass. 

In [1]:
%%capture
!conda install -c pytorch faiss-cpu=1.8.0 -y

Retrieving notices: ...working... done
Channels:
 - pytorch
 - rapidsai
 - nvidia
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.1.2

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - faiss-cpu=1.8.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    faiss-cpu-1.8.0            |py3.10_h9d89a2e_0_cpu         4.4 MB  pytorch
    libfaiss-1.8.0             |   hf65b397_0_cpu         5.2 MB  pytorch
    mkl-2023.2.0               |   h84fe81f_50496       156.8 MB  conda-forge
    ------------------------------------------------------------
                                           Total:       166.3 MB

The following NEW packages will be INSTALLED

In [5]:
# %%capture
# !conda install -c pytorch -c nvidia faiss-gpu=1.8.0 -y

In [2]:
%%capture
!pip install transformers==4.38.2
# !pip install accelerate==0.27.2
!pip install datasets==2.18.0 # Fix issue numpy attributes error of RagRetriever
# !pip install peft==0.9.0
# !pip install bitsandbytes==0.42.0

In [4]:
import os
import torch
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
login(token=user_secrets.get_secret("HUGGINGFACE_TOKEN"))


os.environ["MODEL_NAME"] = "facebook/rag-token-base"
os.environ["DATASET"] = ""

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic=True
    # https://github.com/huggingface/transformers/issues/28731
    torch.backends.cuda.enable_mem_efficient_sdp(False)
    device='cuda'
else:
    device='cpu'

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
# !nvcc --version

/bin/bash: nvcc: command not found


In [5]:
from transformers import AutoTokenizer

tokenizer=AutoTokenizer.from_pretrained(os.getenv("MODEL_NAME"))
tokenizer

config.json:   0%|          | 0.00/4.55k [00:00<?, ?B/s]

(…)_encoder_tokenizer/tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

question_encoder_tokenizer/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)ncoder_tokenizer/special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.


(…)enerator_tokenizer/tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

generator_tokenizer/vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

generator_tokenizer/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)erator_tokenizer/special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizerFast'.


In [6]:
from transformers import RagRetriever

retriever=RagRetriever.from_pretrained(os.getenv("MODEL_NAME"), index_name="exact", use_dummy_dataset=True)
retriever

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

Generating train split: 0 examples [00:00, ? examples/s]

/opt/conda/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for wiki_dpr contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wiki_dpr
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [8]:
from transformers import RagModel

engine=RagModel.from_pretrained(os.getenv("MODEL_NAME"), retriever=retriever, device=device)
engine

pytorch_model.bin:   0%|          | 0.00/2.06G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


RagModel(
  (question_encoder): DPRQuestionEncoder(
    (question_encoder): DPREncoder(
      (bert_model): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1, inplace=False)
                )
                (output): BertSelf

In [9]:
token_input=tokenizer("How many poeple live in Melbourne?", return_tensors="pt")
print(token_input)

{'input_ids': tensor([[  101,  2129,  2116, 18922, 10814,  2444,  1999,  4940,  1029,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [14]:
outputs=engine(input_ids=token_input["input_ids"])
outputs

RetrievAugLMOutput(logits=tensor([[[ 10.3162,  -1.2968,   8.5875,  ...,  -2.5577,  -2.9331,   6.1391],
         [ 10.3162,  -1.2968,   8.5875,  ...,  -2.5577,  -2.9331,   6.1391],
         [ -8.5947,  -1.9153,   3.2866,  ...,   0.9830,  -0.8535,   4.1230],
         ...,
         [-17.6546,  -0.6217,  11.0146,  ...,   4.9040,   2.6374,   5.9604],
         [-17.8028,  -0.5770,  10.9866,  ...,   5.0103,   2.6565,   6.1924],
         [-17.6727,  -0.5310,  10.5987,  ...,   4.9869,   2.5776,   6.3321]],

        [[  9.8321,  -1.5344,   8.6248,  ...,  -3.3438,  -3.7547,   6.3309],
         [  9.8321,  -1.5344,   8.6248,  ...,  -3.3438,  -3.7547,   6.3309],
         [-10.6821,  -4.2749,   6.2310,  ...,  -4.7471,  -5.0958,   3.2340],
         ...,
         [-16.6555,  -0.5150,  11.3066,  ...,   5.0081,   3.3283,   6.6945],
         [-17.0055,  -0.5060,  11.2911,  ...,   5.0653,   3.3135,   6.7807],
         [-17.4794,  -0.4665,  10.8806,  ...,   5.1103,   3.3115,   6.8058]],

        [[ 10.6466

In [12]:
tokenizer.batch_decode()

2024-03-11 01:22:47.272117: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 01:22:47.272370: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 01:22:47.439110: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TypeError: argument 'ids': 'dict' object cannot be converted to 'Sequence'

# Reference List

* https://www.promptingguide.ai/research/rag
* https://blog.gopenai.com/retrieval-augmented-generation-rag-585aa903d6bd
* https://github.com/ray-project/ray
* https://github.com/facebookresearch/faiss/blob/main/INSTALL.md
* https://lightning.ai/docs/pytorch/stable/